In [1]:
import pandas as pd
import asyncio, json
from aiohttp import ClientSession

df = pd.read_excel("prices3.xlsx")
df.columns = [c.lower().strip() for c in df.columns]
df["sku"] = df["sku"].astype(str)


Sku_ProductId_API  = "https://ksa-api.boutiqaat.com/searchplus/rest/V2/global/suggest"
ProductId_Price_API  = "https://ksa-api.boutiqaat.com/searchplus/rest/V2/productinfo?useMultiConfigResponse=true"

HEADERS = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "origin": "https://www.boutiqaat.com",
    "referer": "https://www.boutiqaat.com/",
    "user-agent": "Mozilla/5.0",
    "param": '{"gender":"4194","app_version":"8.0.0","platform":"web","device_type":"desktop"}'
}

BASE_PAYLOAD = {
    "countryCodeAndLanguage": "kw_en",
    "numberOfRecords": 20,
    "featuredOnly": None,
    "searchString": ""
}


HEADERS_2 = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "origin": "https://www.boutiqaat.com",
    "referer": "https://www.boutiqaat.com/",
    "user-agent": "Mozilla/5.0",
    "param": '{"gender":"4194","app_version":"8.0.0","platform":"web","device_type":"desktop"}'
}

BASE_PAYLOAD_2 = {
    "productId": "", #this should be changed
    "ruleId": "",
    #"slug": "1906-sneakers-grey-11/p/", #i don't think we need the slug
    "celebrityId": "0",
    "optionId": "",
    "countryCodeAndLanguage": "kw_en"
}
async def fectch_productId(session, sku):
    payload = dict(BASE_PAYLOAD)
    payload["searchString"] = sku
    async with session.post(Sku_ProductId_API, headers=HEADERS, json=payload) as r:
        j = await r.json(content_type=None)
    try:
        #print(j["data"][0]["data"][0]["displayTextAr"])
        return j["data"][0]["data"][0]["productId"]
    except:
        return None

async def fetch_price(session, productId):
    payload = dict(BASE_PAYLOAD_2)
    payload["productId"] = productId
    async with session.post(ProductId_Price_API, headers=HEADERS, json=payload) as r:
        j = await r.json(content_type=None)
    try:
        return j[0]["final_price_with_tax"]
    except:
        return None
None

async def refresh_price(session, productId):
    url = f"https://ksa-api.boutiqaat.com/cdc/rest/V1/realtime/{productId}"
    async with session.get(url, ssl=False) as r:
        text = await r.text()
        # API returns plain text like "success"
        if "success" in text.lower():
            return "success"
        return "failed"

async def process_row(session, sku, expected_price):
    productId = await fectch_productId(session, sku)
    if not productId:
        return {
            "sku": sku,
            "excel_price": expected_price,
            "web_price": None,
            "match": None,
            "retried": False,
            "refresh_status": "failed"
        }

    # --- First attempt ---
    web_price = await fetch_price(session, productId)
    web_price = float(web_price) if web_price else None
    match = (web_price == expected_price) if web_price is not None else None

    # --- Retry if mismatch ---
    retried = False
    refresh_status = "-"

    if match is False:
        retried = True
        refresh_status = await refresh_price(session, productId)

        if refresh_status == "success":
            # optional small pause for backend to update
            await asyncio.sleep(0.5)
            web_price = await fetch_price(session, productId)
            web_price = float(web_price) if web_price else None
            match = (web_price == expected_price) if web_price is not None else None

    return {
        "sku": sku,
        "excel_price": expected_price,
        "web_price": web_price,
        "match": match,
        #"retried": retried,
        "refresh_status": refresh_status
    }

# Run
async def main(df):
    async with ClientSession() as s:
        #df = df.head(10)
        return await asyncio.gather(*(
            process_row(s, sku, float(price))
            for sku, price in zip(df["sku"], df["special price"])
        ))

out = await main(df)
results = pd.DataFrame(out)

# Save
results.to_excel("comparison_results.xlsx", index=False)
results.to_csv("comparison_results.csv", index=False)

results.head(50)

,sku,excel_price,web_price,match,refresh_status
0,ORL-00006341,6.600,6.60,True,-
1,ORL-00006341,6.601,6.60,False,success
2,I-00000190608,15.000,15.00,True,-
3,I-00000190608,15.001,15.00,False,success
4,ORL-00006239,22.500,18.00,False,success
5,ORL-00006239,22.501,18.00,False,success
6,ORL-00004724,17.130,17.13,True,-
7,ORL-00004724,17.131,17.13,False,success
8,MU-00022119,8.500,8.50,True,-
9,MU-00022119,8.501,8.50,False,success
